In [38]:
#import packages
import pandas as pd

In [39]:
#import lyrics dataset
lyric = pd.read_csv("Eurovision-lyrics-Trackid_csv_edited.csv",sep=";")

In [40]:
#import audio features
audio = pd.read_csv("Spotify_Eurovision_extended.csv")

In [41]:
#merging audio and lyrics
df_merged = pd.merge(lyric,audio,on = "TrackID",how="left")

In [42]:
#convert data types to the most suitable
df_merged = df_merged.convert_dtypes()

In [43]:
#Convert Pl. to numeric
df_merged["Pl."] = pd.to_numeric(df_merged["Pl."], errors='coerce')

In [44]:
#Convert Sc. to numeric
df_merged["Sc."] = pd.to_numeric(df_merged["Sc."], errors='coerce')

In [45]:
#Drop duplicate columns
df_merged = df_merged.drop(columns = ["Unnamed: 0_x","Unnamed: 0_y","#"])

In [46]:
#Check if there any duplicated TrackID
df_merged[df_merged["TrackID"].duplicated()]["TrackID"]

91      <NA>
93      <NA>
94      <NA>
99      <NA>
103     <NA>
        ... 
1010    <NA>
1032    <NA>
1163    <NA>
1185    <NA>
1199    <NA>
Name: TrackID, Length: 274, dtype: string

In [47]:
#Check languages imaginary 
df_merged[df_merged["Lyrics translation"].isnull()]

,Country,#.1,Artist,Song,Language,Pl.,Sc.,Eurovision_Number,Year,TrackID,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,key,mode,duration_ms,Popularity
912,Belgium,45,Urban Trad,Sanomi,Imaginary,2.0,165.0,48,2003,788II2WAQ8lOuW1CSQ9oHp,...,0.0303,0.188,0.0,0.137,0.638,85.003,7,0,247507,16
1076,Belgium,50,Ishtar,O Julissi,Imaginary,NaN,NaN,53,2008,2eE5Q1xHwy5aVAB2RAQwJG,...,0.0404,0.874,0.000002,0.22,0.731,135.918,7,1,154387,14


In [48]:
#Duplicate english lyrics to traslated lyrics column
for row in range(0,1644):
    if pd.isna (df_merged.iloc[row,13]):
        pass
    elif df_merged.iloc[row,13] == "English":
        df_merged.iloc[row,13] = df_merged.iloc[row,12]

In [49]:
#Drop Null values in Pl. (Songs without rank)
df_merged = df_merged.dropna(subset=["Pl."])

In [50]:
#min-max normalization of scores within same year
standard_score = lambda x: (x - x.min()) / (x.max()-x.min())
df_merged.insert(8, 'standard_score', df_merged.groupby(["Eurovision_Number"])['Sc.'].transform(standard_score))

In [51]:
#change first year winning song to 1
df_merged.iloc[0,8] = 1

In [52]:
#Normalize audio features (to range 0-1)
cols_to_norm = ['loudness','tempo']
df_merged[cols_to_norm] = df_merged[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [53]:
#One hot encoding Key column 
df_merged = pd.get_dummies(df_merged, columns=['key'], drop_first=False)

In [54]:
#Drop song with imaginary lyrics
df_merged = df_merged.dropna(subset=["Lyrics translation"])

In [55]:
#Drop songs without TrackID column (no audio features)
df_merged = df_merged.dropna(subset=["TrackID"]) 

In [56]:
df_merged

,Country,#.1,Artist,Song,Language,Pl.,Sc.,Eurovision_Number,standard_score,Year,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
8,Switzerland,-,Lys Assia (2),Refrain,French,1.0,NaN,1,1.000000,1956,...,0,0,0,0,0,0,0,0,0,0
14,Belgium,2,Bobbejaan Schoepen,Straatdeuntje,Dutch,8.0,5.0,2,0.071429,1957,...,0,0,0,0,0,0,0,0,1,0
15,Luxembourg,2,Danièle Dupré,Amours mortes (Tant de peine),French,4.0,8.0,2,0.178571,1957,...,0,0,0,0,0,1,0,0,0,0
16,United Kingdom,1,Patricia Bredin,All,English,7.0,6.0,2,0.107143,1957,...,0,0,0,1,0,0,0,0,0,0
17,Italy,2,Nunzio Gallo,Corde della mia chitarra,Italian,6.0,7.0,2,0.142857,1957,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,Sweden,60,Tusse,Voices,English,14.0,109.0,65,0.208015,2021,...,0,0,0,0,0,0,0,0,1,0
1640,Switzerland,61,Gjon's Tears,Tout l'Univers,French,3.0,432.0,65,0.824427,2021,...,0,0,0,0,0,0,0,1,0,0
1641,Netherlands,61,Jeangu Macrooy,Birth Of A New Age,English/Sranan Tongo,23.0,11.0,65,0.020992,2021,...,0,0,0,0,0,0,0,0,0,0
1642,Ukraine,16,Go_A,Shum,Ukrainian,5.0,364.0,65,0.694656,2021,...,0,0,1,0,0,0,0,0,0,0


In [57]:
#Reset index in merged dataset
df_merged = df_merged.reset_index(drop=True)

In [58]:
#Check the last version
df_merged

,Country,#.1,Artist,Song,Language,Pl.,Sc.,Eurovision_Number,standard_score,Year,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,Switzerland,-,Lys Assia (2),Refrain,French,1.0,NaN,1,1.000000,1956,...,0,0,0,0,0,0,0,0,0,0
1,Belgium,2,Bobbejaan Schoepen,Straatdeuntje,Dutch,8.0,5.0,2,0.071429,1957,...,0,0,0,0,0,0,0,0,1,0
2,Luxembourg,2,Danièle Dupré,Amours mortes (Tant de peine),French,4.0,8.0,2,0.178571,1957,...,0,0,0,0,0,1,0,0,0,0
3,United Kingdom,1,Patricia Bredin,All,English,7.0,6.0,2,0.107143,1957,...,0,0,0,1,0,0,0,0,0,0
4,Italy,2,Nunzio Gallo,Corde della mia chitarra,Italian,6.0,7.0,2,0.142857,1957,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1071,Sweden,60,Tusse,Voices,English,14.0,109.0,65,0.208015,2021,...,0,0,0,0,0,0,0,0,1,0
1072,Switzerland,61,Gjon's Tears,Tout l'Univers,French,3.0,432.0,65,0.824427,2021,...,0,0,0,0,0,0,0,1,0,0
1073,Netherlands,61,Jeangu Macrooy,Birth Of A New Age,English/Sranan Tongo,23.0,11.0,65,0.020992,2021,...,0,0,0,0,0,0,0,0,0,0
1074,Ukraine,16,Go_A,Shum,Ukrainian,5.0,364.0,65,0.694656,2021,...,0,0,1,0,0,0,0,0,0,0


In [59]:
df_merged.iloc[:,8:].head(20)

,standard_score,Year,TrackID,Host_Country,Host_City,Lyrics,Lyrics translation,playlistID,TrackName,SampleURL,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,1.000000,1956,3ijeTyo5ggRvM0DTfZflBB,Switzerland,Lugano,[Intro] (Refrain d'amour) [Couplet 1] Refrain...,"Chorus of love, or or or or Chorus, color of t...",39mNowYJVhim6TyP6IMipP,Refrain,https://p.scdn.co/mp3-preview/efc8bb73b06c9c46...,...,0,0,0,0,0,0,0,0,0,0
1,0.071429,1957,3BM3vLAbeECkvyWWS3qdGK,West Germany,Frankfurt,Langs de straten danst er een deuntje En ik bl...,"Along the streets, a tune is dancing And for a...",39mNowYJVhim6TyP6IMipP,"Straatdeuntje (Original, 1957)",https://p.scdn.co/mp3-preview/98fcf347c9ccce2b...,...,0,0,0,0,0,0,0,0,1,0
2,0.178571,1957,3g4Pmtk3Up0NCYRFqcEPsQ,West Germany,Frankfurt,Toi que j'aime Tu m'as fait tant de peine Mais...,"Dead loves You, that I love You made me so muc...",39mNowYJVhim6TyP6IMipP,Tant De Peine,https://p.scdn.co/mp3-preview/9df9ff351eaf4d21...,...,0,0,0,0,0,1,0,0,0,0
3,0.107143,1957,4Sqs1vBhEoo3dfqMvUPhgh,West Germany,Frankfurt,All the golden dreams of yesterday All the fra...,All the golden dreams of yesterday All the fra...,39mNowYJVhim6TyP6IMipP,All,https://p.scdn.co/mp3-preview/a58ef4d56caba3f0...,...,0,0,0,1,0,0,0,0,0,0
4,0.142857,1957,00N2j8JtP4Xj7a9ELwqerR,West Germany,Frankfurt,È tornata L'hanno accolta le stesse cose L'han...,She has returned The same things welcomed her ...,39mNowYJVhim6TyP6IMipP,Corde della mia chitarra,https://p.scdn.co/mp3-preview/bd352d6d5fffb21a...,...,0,0,0,0,0,1,0,0,0,0
5,0.000000,1957,0mLmt6DBB2QITlKPXLwSxn,West Germany,Frankfurt,"Wohin, kleines Pony, wollen wir reiten? Wohin ...","Where, little pony, do we want to ride to? Whe...",39mNowYJVhim6TyP6IMipP,Wohin Kleines Pony,https://p.scdn.co/mp3-preview/28d5caaad8d41904...,...,0,0,0,0,0,0,0,0,0,0
6,1.000000,1957,3ZRIa6YJiMXzJ071bEcG3V,West Germany,Frankfurt,Zit niet zo suf Met die eeuwige krant Gaap nie...,Don’t be so drowsy with that eternal paper Do...,39mNowYJVhim6TyP6IMipP,Net Als Toen,<NA>,...,0,0,0,0,0,0,1,0,0,0
7,0.178571,1957,21Rc1BtK7weDmW2caDLWus,West Germany,Frankfurt,"Telefon, Telefon Wenn dein Ruf erklingt Frag i...","Telephone, telephone Whenever I hear your call...",39mNowYJVhim6TyP6IMipP,Telefon Telefon,https://p.scdn.co/mp3-preview/af894f8f16f4f276...,...,0,0,0,0,0,1,0,0,0,0
8,0.500000,1957,1pMDLjdoyAOfucSkDWSVJQ,West Germany,Frankfurt,"Si vous aimez la nature Les lilas, les petits ...","If you love nature Lilacs, little birds Muscad...",39mNowYJVhim6TyP6IMipP,La belle amour,https://p.scdn.co/mp3-preview/14a04cc2ebf98f2c...,...,0,0,0,0,0,0,0,0,1,0
9,0.250000,1957,1Sll2uSqvRxvKbhOTSVs4O,West Germany,Frankfurt,"Farvel da, min tøs Og hold dig nu kvik Jeg til...","Goodbye then, my lass, stay well I confess, my...",39mNowYJVhim6TyP6IMipP,Skibet Skal Sejle I Nat,<NA>,...,0,1,0,0,0,0,0,0,0,0


In [60]:
#Saving as csv file
df_merged.to_csv(r'eurovision_preprocessed.csv')